In [503]:
class Matrix:
    
    def __init__(self, data):
        if self.check_integrity(data) == False:
            raise Exception('Matrix is incorrectly configured with row, column, or data type mismatch. Please provide a list of lists containing only ints or floats of consistent dimensions.')
        self.data = data
        self.rows = len(data)
        self.cols = len(data[0])
        self.shape = (self.rows, self.cols)
        self.size = self.rows * self.cols

    def check_integrity(self, data) -> bool:
        '''
        Args - data (list): ...
        Returns - bool
        '''
        # Check if provided matrix is a list
        if isinstance(data, list) == False:
            return False
        
        # Check if provided matrix is a list of lists
        checks = [isinstance(x, list) == False for x in data]
        if True in checks:
            return False
        
        # Check if provided matrix has consistent dimensions
        ncols = [len(x) for x in data]
        if sum(ncols) / len(ncols) != ncols[0]:
            return False
        
        # Check if provided matrix contains values of type int or float only
        for i in range(len(data)):
            checks = [isinstance(x, (int, float)) == False for x in data[i]]
            if sum(checks) > 0:
                return False

    def assign(self, row, col, value) -> None:
        if isinstance(value, (int, float)) == False:
            raise Exception('Value must be of type int or float only.')
        self.data[row][col] = value
    
    # TODO
    def fill(self, row_range, col_range, value) -> None:
        if isinstance(value, (int, float)) == False:
            raise Exception('Value must be of type int or float only.')
        self.data[row_range][col_range] = value
    
    def retrieve(self, row, col) -> (int, float):
        return self.data[row][col]

    def diagonal(self) -> object:
        matrix = []
        for i in range(self.rows):
            if i < self.cols:
                matrix.append([self.data[i][i]])
        return Matrix(matrix)

    def transpose(self) -> object:
        matrix, row = [], []
        for j in range(self.cols):
            for i in range(self.rows):
                row.append(self.data[i][j])
            matrix.append(row)
            row = []
        return Matrix(matrix)

    # TODO
    def inverse(self) -> object:
        if self.shape[0] != self.shape[1] and self.shape[0] < 2:
            raise Exception('Matrix is not invertible as it is a non-square matrix. Please provide a square matrix with dimensions greater than or equal to (2, 2).')
        matrix = [[1], [0]]
        return Matrix(matrix)

    # TODO
    def determinant(self) -> float:
        if self.shape[0] != self.shape[1] and self.shape[0] < 2:
            raise Exception('Matrix determinant cannot be derived as it is a non-square matrix. Please provide a square matrix with dimensions greater than or equal to (2, 2).')
        return 0

    def __add__(self, other) -> object:
        if other.shape != self.shape:
            raise Exception('Matrix dimensions are mismatched ({} != {}). Please add (2) matrices of the same dimensions.'.format(self.shape, other.shape))
        data, output = self.data, []
        for i in range(self.rows):
            output.append([])
            for j in range(self.cols):
                output[i].append(self.data[i][j] + other.data[i][j])
        self.data = data
        return Matrix(output)

    def __sub__(self, other):
        if other.shape != self.shape:
            raise Exception('Matrix dimensions are mismatched ({} != {}). Please subtract (2) matrices of the same dimensions.'.format(self.shape, other.shape))
        data, output = self.data, []
        for i in range(self.rows):
            output.append([])
            for j in range(self.cols):
                output[i].append(self.data[i][j] - other.data[i][j])
        self.data = data
        return Matrix(output)

    def __lmul__(self, scalar) -> object:
        if isinstance(scalar, (int, float)) == False:
            raise Exception('Scalar must be of type int or float only.')
        data, output = self.data, []
        for i in range(self.rows):
            output.append([])
            for j in range(self.cols):
                output[i].append(self.data[i][j] * scalar)
        self.data = data
        return Matrix(output)

    def __rmul__(self, scalar) -> object:
        if isinstance(scalar, (int, float)) == False:
            raise Exception('Scalar must be of type int or float only.')
        data, output = self.data, []
        for i in range(self.rows):
            output.append([])
            for j in range(self.cols):
                output[i].append(self.data[i][j] * scalar)
        self.data = data
        return Matrix(output)

    def __mul__(self, scalar) -> object:
        if isinstance(scalar, (int, float)) == False:
            raise Exception('Scalar must be of type int or float only.')
        data, output = self.data, []
        for i in range(self.rows):
            output.append([])
            for j in range(self.cols):
                output[i].append(self.data[i][j] * scalar)
        self.data = data
        return Matrix(output)

    # TODO
    #def __mul__(self, other):
    #    pass

    # TODO
    #def cross(self, other):
    #    pass

    # TODO
    #def dot(self, other):
    #    pass

    def __repr__(self) -> str:
        matrix = str(self.data).replace('],', '],\n ')
        return "<class: 'Matrix'>\nDimensions: {} row(s) x {} column(s)\n {}\nSize: {}".format(self.rows, self.cols, matrix, self.size)

In [504]:
A = Matrix([[1, 0, 0, 0], [0, 2, 2, 0], [0, 0, 3, 0], [0, 0, 0, 4]])

In [505]:
A

<class: 'Matrix'>
Dimensions: 4 row(s) x 4 column(s)
 [[1, 0, 0, 0],
  [0, 2, 2, 0],
  [0, 0, 3, 0],
  [0, 0, 0, 4]]
Size: 16

In [506]:
print(A.shape)
print(A.size)

(4, 4)
16


In [507]:
A.transpose()

<class: 'Matrix'>
Dimensions: 4 row(s) x 4 column(s)
 [[1, 0, 0, 0],
  [0, 2, 0, 0],
  [0, 2, 3, 0],
  [0, 0, 0, 4]]
Size: 16

In [508]:
A+A

<class: 'Matrix'>
Dimensions: 4 row(s) x 4 column(s)
 [[2, 0, 0, 0],
  [0, 4, 4, 0],
  [0, 0, 6, 0],
  [0, 0, 0, 8]]
Size: 16

In [509]:
A-A

<class: 'Matrix'>
Dimensions: 4 row(s) x 4 column(s)
 [[0, 0, 0, 0],
  [0, 0, 0, 0],
  [0, 0, 0, 0],
  [0, 0, 0, 0]]
Size: 16

In [510]:
A

<class: 'Matrix'>
Dimensions: 4 row(s) x 4 column(s)
 [[1, 0, 0, 0],
  [0, 2, 2, 0],
  [0, 0, 3, 0],
  [0, 0, 0, 4]]
Size: 16

In [511]:
A * 2

<class: 'Matrix'>
Dimensions: 4 row(s) x 4 column(s)
 [[2, 0, 0, 0],
  [0, 4, 4, 0],
  [0, 0, 6, 0],
  [0, 0, 0, 8]]
Size: 16

In [512]:
A.transpose() * 2

<class: 'Matrix'>
Dimensions: 4 row(s) x 4 column(s)
 [[2, 0, 0, 0],
  [0, 4, 0, 0],
  [0, 4, 6, 0],
  [0, 0, 0, 8]]
Size: 16

In [513]:
2 * A - A + 3 * A

<class: 'Matrix'>
Dimensions: 4 row(s) x 4 column(s)
 [[4, 0, 0, 0],
  [0, 8, 8, 0],
  [0, 0, 12, 0],
  [0, 0, 0, 16]]
Size: 16

In [514]:
A

<class: 'Matrix'>
Dimensions: 4 row(s) x 4 column(s)
 [[1, 0, 0, 0],
  [0, 2, 2, 0],
  [0, 0, 3, 0],
  [0, 0, 0, 4]]
Size: 16

In [515]:
A.diagonal()

<class: 'Matrix'>
Dimensions: 4 row(s) x 1 column(s)
 [[1],
  [2],
  [3],
  [4]]
Size: 4

In [516]:
B = Matrix([[1], [0]])

In [517]:
B

<class: 'Matrix'>
Dimensions: 2 row(s) x 1 column(s)
 [[1],
  [0]]
Size: 2

In [518]:
B.diagonal()

<class: 'Matrix'>
Dimensions: 1 row(s) x 1 column(s)
 [[1]]
Size: 1

In [519]:
B.assign(0, 0, 5)

In [520]:
B

<class: 'Matrix'>
Dimensions: 2 row(s) x 1 column(s)
 [[5],
  [0]]
Size: 2

In [521]:
A.fill(0:4, 0:1, 7)

SyntaxError: invalid syntax (<ipython-input-521-2b04fd954c7e>, line 1)

In [525]:
A.retrieve(2, 2)

3